# ALI Performance Tests on Weaver

## Introduction
Currently testing the Greenland Ice Sheet (GIS) in Albany Land Ice (ALI) using Nvidia Tesla V100 GPUs on weaver. 
Note: Waterman was moved to a different network and renamed weaver. The last waterman test was executed on 6/16/2020. 

### Architectures: 
| Name | Weaver (P9/V100) |
|---|---|
| CPU | Dual-socket IBM POWER9 |
| GPU | Nvidia Tesla V100 |
| Cores/Node | 40 |
| Threads/Core | 8 |
| GPUs/Node | 4 |
| Memory/Node | 319 GB |
| Interconnect | Mellanox EDR IB (100 GB/s) |
| Compiler | gcc 7.2.0 |
| GPU Compiler | cuda 10.1.105 |
| MPI | openmpi 4.0.1 |

### Cases: 
| Case Name  | Number of Processes (np) | Description |
|---|---|---|
| green-1-10km_ent_fea_mem_tet | 8 | Unstructured 1-10km GIS, Enthalpy problem, finite element assembly only, memoization, tetrahedron |
| green-1-7km_fea_1ws | 8 | Unstructured 1-7km GIS, finite element assembly only, single workset |
| green-1-7km_fea_mem | 8 | Unstructured 1-7km GIS, finite element assembly only, memoization |
| green-1-7km_muk_ls_mem | 8 | Unstructured 1-7km GIS, MueLu w/ kokkos and line smoothing, memoization |
| green-3-20km_vel_fea_mem_tet | 8 | Unstructured 3-20km GIS, Velocity problem, finite element assembly only, memoization, tetrahedron |
| green-3-20km_vel_fea_mem_wdg | 8 | Unstructured 3-20km GIS, Velocity problem, finite element assembly only, memoization, wedge |
| green-3-20km_ent_fea_mem_tet | 8 | Unstructured 3-20km GIS, Enthalpy problem, finite element assembly only, memoization, tetrahedron |
| green-3-20km_ent_fea_mem_wdg | 8 | Unstructured 3-20km GIS, Enthalpy problem, finite element assembly only, memoization, wedge |

### Timers: 
| Timer Name | Level | Description |
|---|---|---|
| Albany Total Time | 0 | Total wall-clock time of simulation |
| Albany: Setup Time | 1 | Preprocess |
| Albany: Total Fill Time | 1 | Finite element assembly |
| Albany Fill: Residual | 2 | Residual assembly |
| Albany Residual Fill: Evaluate | 3 | Compute the residual, local/global assembly |
| Albany Residual Fill: Export | 3 | Update global residual across MPI ranks |
| Albany Fill: Jacobian | 2 | Jacobian assembly |
| Albany Jacobian Fill: Evaluate | 3 | Compute the Jacobian, local/global assembly |
| Albany Jacobian Fill: Export | 3 | Update global Jacobian across MPI ranks |
| NOX Total Preconditioner Construction | 1 | Construct Preconditioner |
| NOX Total Linear Solve | 1 | Linear Solve |

In [ ]:
import datetime as dt
import glob
import numpy as np
import pandas as pd
import json
import multiprocessing
import sys

import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

# Import scripts
sys.path.insert(0,'kcshan-perf-analysis')
from json2timeline import json2dataframe
from models import single_ts_chgpts
from basicstats import add_regime_stats
from basicstats import trimmed_stats
from utils import *
from scipy.stats import t as tdist

In [ ]:
hide_code_button()

In [ ]:
# Enable offline plot export
init_notebook_mode(connected=True)

## Specifications

In [ ]:
# Load configuration file
with open('config.json') as jf:
    config = json.load(jf)
check_config(config)
for key,val in config.items():
        exec(key + '=val')

# Extract file names and collect data
files = glob.glob(json_regex)
df = json2dataframe(files, cases, names, timers, metadata)
if len(df) == 0:
    raise RuntimeError('No data found; check json directory')

# Log-transform the data before modeling
xform = lambda x: np.log(x)
inv_xform = lambda x: np.exp(x)

# Add other metrics to name list
names.append('max host memory')
names.append('max kokkos memory')
metadata.remove('max host memory')
metadata.remove('max kokkos memory')

# Filter data by date if desired
#import datetime as dt
#df = df[df['date'] < dt.datetime.strptime('20211201', '%Y%m%d')]
#df = df[df['date'] > dt.datetime.strptime('20210601', '%Y%m%d')]

In [ ]:
print('Test cases:')
[print('  '+c) for c in cases]
print('Metrics:')
[print('  '+n) for n in names]
print('Metadata:')
[print('  '+m) for m in metadata]
print("Model threshold: %f" % threshold)

## Performance Timelines

In [ ]:
np.seterr(all='raise')

def regime_diff_ts(y, changePts, std_error=False, alpha=0.01):
    '''
    Given a timeseries y and a set of changepoints, return three timeseries with the
    same length as y, containing the mean difference and upper/lower bounds between changepoints
    Inputs:
        y        : time series input
        changePts: sorted list of indices of y which are changepoints, 
                     where changePts[0] = 0
        std_error: whether reported std should be std error of the mean
        alpha    : if std_error, then this controls the significance level of the bounds
    '''
    y = make_numpy(y)
    n = len(y)
    m = len(changePts)
    meandiff = np.zeros_like(y)
    upper = np.zeros_like(y)
    lower = np.zeros_like(y)
    for i in range(1,m):
        a = changePts[i-1]
        b = changePts[i]
        c = changePts[i+1] if i < m-1 else n
        
        y1mean, y1var = trimmed_stats(y[a:b], var=True)
        n1 = len(y[a:b])
        y2mean, y2var = trimmed_stats(y[b:c], var=True)
        n2 = len(y[b:c])
        meandiff[b:c] = np.abs(y1mean-y2mean)
        if (n1 == 1 and n2 == 1):
            continue
        
        rss = (n1-1)*y1var + (n2-1)*y2var
        std = np.sqrt(rss/(n1+n2-2)) + 1e-8
        if std_error:
            std /= np.sqrt(n1*n2/(n1+n2))
            tcrit = tdist.isf(alpha/2, df=n1+n2-2)
        else:
            tcrit = 2
        upper[b:c] = meandiff[b:c] + tcrit*std
        lower[b:c] = meandiff[b:c] - tcrit*std
    return meandiff, upper, lower

def add_regime_diff_stats(df, changePts, std_error=False, alpha=0.01):
    '''
    Take a dataframe with a 'time' column, and append the output of regime_diff_ts
    '''
    meandiff, upper, lower = regime_diff_ts(df['time'], changePts, std_error, alpha)
    temp = {'meandiff': meandiff, 'meandiff_upper': upper, 'meandiff_lower': lower}
    return pd.concat((df, pd.DataFrame(temp)), axis=1)


# Find changepoints and format data to work nicely with plots
seqs = {case:{} for case in cases}
most_recent = df['date'].max()
events = {}
pool = multiprocessing.Pool(4)

print('Finding changepoints:')
for case in cases:
    print(case)
    
    # Add time data to seqs
    for name in names:
        cols = ['date', name] + list(metadata)
        data = df.loc[df['case']==case, cols].dropna(subset=[name])
        data.reset_index(drop=True, inplace=True)
        data.rename(columns={name:'time'}, inplace=True)
        data['time'] = xform(data['time'])
        seqs[case][name] = data
    
    # Detect changepoints  
    pool_inputs = [(k, v, threshold) for k,v in seqs[case].items()]
    chgpts = dict(pool.map(single_ts_chgpts, pool_inputs))
    
    for name in names:
        # Calculate mean/99%CI between changepoints
        seqs[case][name] = add_regime_stats(seqs[case][name], chgpts[name], std_error=True, alpha=0.01)
        seqs[case][name]['means'] = inv_xform(seqs[case][name].pop('mean'))
        seqs[case][name]['means_lower'] = np.nan_to_num(inv_xform(seqs[case][name].pop('lower')))
        seqs[case][name]['means_upper'] = np.nan_to_num(inv_xform(seqs[case][name].pop('upper')))
        
        # Calculate mean/std between changepoints
        seqs[case][name] = add_regime_stats(seqs[case][name], chgpts[name])
        
        # Calculate meandiff/99%CI between changepoints
        seqs[case][name] = add_regime_diff_stats(seqs[case][name], chgpts[name], std_error=True, alpha=0.01)
        seqs[case][name]['meandiff'] = inv_xform(seqs[case][name]['meandiff'])
        seqs[case][name]['meandiff_lower'] = inv_xform(seqs[case][name]['meandiff_lower'])
        seqs[case][name]['meandiff_upper'] = inv_xform(seqs[case][name]['meandiff_upper'])
        
        # Build dictionary of changepoints
        for d in seqs[case][name]['date'].iloc[chgpts[name]]:
            events.setdefault(d, {}).setdefault(case, []).append(name)
clear_output()

# Sort and print results
events = {k:events[k] for k in sorted(events.keys())}
print('Events in the most recent %d days:' % recency)
recent_events = print_events(events, most_recent, recency)

In [ ]:
lines = ['time', 'mean', 'upper', 'lower']
colors = ['blue', 'red', 'red', 'red']
modes = ['markers', 'lines', 'lines', 'lines']
dashes = ['solid', 'solid', 'dot', 'dot']

fig = go.Figure()
# Create series on plot
for line, color, mode, dash in zip(lines, colors, modes, dashes):
    for c in cases:
        if line == 'time':
            fig.add_trace(go.Scatter(
                x=seqs[c][names[0]]['date'],
                y=inv_xform(seqs[c][names[0]][line]),
                mode=mode,
                line = dict(color=color, dash=dash, width=1.5),
                name=line,
                visible=True if c==cases[0] else False,
                customdata=seqs[c][names[0]][['date']+['means']+['means_lower']+['means_upper']+['meandiff']+['meandiff_lower']+['meandiff_upper']+list(metadata)],
                hovertemplate=
                "Date: %{customdata[0]}<br>" +
                "Albany commit: %{customdata[8]}<br>" +
                "Trilinos commit: %{customdata[9]}<br>" +
                "Mean (99% CI): %{customdata[1]:.3g} (%{customdata[2]:.3g}, %{customdata[3]:.3g})<br>" +
                "Ratio (99% CI): %{customdata[4]:1.2f} (%{customdata[5]:1.2f}, %{customdata[6]:1.2f})" +
                "<extra></extra>",
            ))
        else:
            fig.add_trace(go.Scatter(
                x=seqs[c][names[0]]['date'],
                y=inv_xform(seqs[c][names[0]][line]),
                mode=mode,
                line = dict(color=color, dash=dash, width=1.5),
                name=line,
                visible=True if c==cases[0] else False,
                hoverinfo='skip'
            ))

changed_cases = {n for v in recent_events.values() for n in v.keys()}

# Test case dropdown
case_options = [dict(
        args=['visible', [True if x==c else False for x in np.tile(cases, len(lines))]],
        label= '*'+c if c in changed_cases else c,
        method='restyle'
    ) for c in cases]
    
# Timer dropdown
name_options = [dict(
        args=[{'x': [seqs[c][n]['date'] for _ in lines for c in cases],
               'y': [inv_xform(seqs[c][n][line]) for line in lines for c in cases],
               'customdata': [seqs[c][n][['date']+['means']+['means_lower']+['means_upper']+['meandiff']+['meandiff_lower']+['meandiff_upper']+list(metadata)].to_numpy()
                              if line == 'time' else None
                              for line in lines for c in cases]}],
        label=n,
        method='restyle'
    ) for n in names]

# Add dropdowns to plot
fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=list(case_options),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,    xanchor="left",
            y=1.15, yanchor="top"
        ),
        go.layout.Updatemenu(
            buttons=list(name_options),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.3, xanchor="left",
            y=1.15, yanchor="top"
        ),
    ],
    margin={'l': 50, 'r': 50, 'b': 200, 't': 50},
    height=600,
    xaxis_title='Simulation Date',
    yaxis_title='Wall-clock Time (s) or Memory (MiB)'
)

iplot(fig)

### Plot of wall-clock times or memory for nightly runs
Changepoints are estimated using a generalized likelihood ratio method on each timer, and then merged over all timers for a given test case. 
* Blue markers: recorded wall-clock time or memory
* Solid red line: average wall-clock time or memory between changepoints
* Dotted red lines: average wall-clock time or memory $\pm$ two standard deviations

#### Plot window controls

* Test case and timer can be selected from the drop-down menus (* denotes recent changes detected)
* Hovering over data points shows various metadata.
  * The mean with upper and lower bounds of a 99% confidence interval.
  * The ratio (speedup/slowdown) between sets identified by changepoints with a 99% confidence interval for the ratio (difference between log mean of the two sets).
  * Note: the confidence interval is based on a t-statistic, so for very small amounts of data (<5), the interval may be very large.
* Clicking on the legend will show/hide individual plot elements
* Click and drag to zoom in; double click to reset zoom

Pollak, Moshe; Siegmund, D. Sequential Detection of a Change in a Normal Mean when the Initial Value is Unknown. Ann. Statist. 19 (1991), no. 1, 394--416. doi:10.1214/aos/1176347990. https://projecteuclid.org/euclid.aos/1176347990

Siegmund, D.; Venkatraman, E. S. Using the Generalized Likelihood Ratio Statistic for Sequential Detection of a Change-Point. Ann. Statist. 23 (1995), no. 1, 255--271. doi:10.1214/aos/1176324466. https://projecteuclid.org/euclid.aos/1176324466

Hawkins, D. M., & Zamba, K. D. (2005). Statistical Process Control for Shifts in Mean or Variance using a Change Point Formulation. Technometrics, 47, 164-173.

Hawkins DM, Qiu P, Kang CW. The changepoint model for statistical process control. Journal of Quality Technology. 2003 Oct 1;35(4):355-366.